# Beaver Tutorial 1: Hello World

Welcome to Beaver! This tutorial will walk you through the basics of collaborative data science with **privacy-preserving Twins**.

## What You'll Learn

1. Setting up Beaver and connecting to a session
2. Creating and sharing data with **Twins**
3. Running computations on remote data
4. Requesting and approving private data access

## Step 1: Import Beaver and Create a Context

First, we import `beaver` and create a **context** (`bv`). The context is your gateway to all Beaver functionality - it lets you:
- Decorate functions with `@bv` to make them privacy-aware
- Access datasets and remote variables
- Manage sessions with peers

In [ ]:
import beaver

# Create the Beaver context - this auto-detects your identity from the session
bv = beaver.ctx()

## Step 2: Connect to Your Session

When you open a notebook from BioVault, it's automatically tied to a **session**. A session is a secure communication channel between you and a peer (another researcher).

**Tip:** You can start a session with yourself for testing! This is useful for learning how Beaver works before collaborating with others.

In [ ]:
# Get the active session - this notebook is tied to a specific session
session = bv.active_session()

# View session info
session

## Step 3: Browse Available Datasets

In Beaver, data is organized into **datasets** published by different users. You can browse what's available:

In [ ]:
# See all available datasets
bv.datasets

**Tip:** In the BioVault app, go to **Network → Datasets** tab to browse datasets visually. You can copy/paste the import code directly from there!

### Creating a Dataset in BioVault App

You can also create datasets using the BioVault app UI:

1. Go to **Data → New Dataset**
2. Give your dataset a name
3. Add files as **Private** (real data) and **Mock** (synthetic data) Twins
4. Click **Save Dataset**
5. Hit **Publish** to make it available to peers

Once published, your dataset will appear here in the Python API!

To access a specific dataset:

In [ ]:
# Access a dataset by owner and name
# Example: dataset = bv.datasets["owner@email.com"]["dataset_name"]

# View what's in the session peer's datasets (if they have any)
print(f"Looking for datasets from: {session.peer}")
try:
    peer_datasets = bv.datasets[session.peer]
    print(peer_datasets)
except KeyError:
    print("No datasets published yet. Let's create one!")

## Step 4: Create Your Own Data (The Twin Pattern)

A **Twin** is the core concept in Beaver. It has two sides:
- **Public side** (mock/synthetic data): Safe to share, used for testing
- **Private side** (real data): Sensitive, requires explicit approval to access

Let's create some example data files first:

In [ ]:
import os

# Create a local data folder
data_dir = "./my_data"
os.makedirs(data_dir, exist_ok=True)

# Create MOCK data (synthetic/public - safe to share)
mock_csv = f"{data_dir}/mock_prices.csv"
with open(mock_csv, 'w') as f:
    f.write("item,units,price\n")
    f.write("apple,2,1.40\n")
    f.write("banana,3,0.75\n")
    f.write("orange,1,1.10\n")

# Create REAL data (private - sensitive)
real_csv = f"{data_dir}/real_prices.csv"
with open(real_csv, 'w') as f:
    f.write("item,units,price\n")
    f.write("apple,10,1.50\n")
    f.write("banana,8,0.80\n")
    f.write("orange,12,1.20\n")

print(f"Created mock data: {mock_csv}")
print(f"Created real data: {real_csv}")

## Step 5: Create a Twin

Now let's wrap this data in a Twin. The Twin keeps track of both versions and ensures the private data is only accessible with explicit approval.

In [ ]:
import pandas as pd
from beaver import Twin

# Load both datasets
mock_df = pd.read_csv(mock_csv)
real_df = pd.read_csv(real_csv)

# Create a Twin with both sides
prices = Twin(
    public=mock_df,    # Safe to share - used for testing
    private=real_df,   # Sensitive - requires approval
    name="prices",
    owner=bv.user      # Your identity
)

# View the Twin
prices

## Step 6: Publish the Twin as a Remote Variable

To share data with your session peer, publish it as a **remote variable**. Only the **public side** is shared - the private data stays with you!

In [ ]:
# Publish to session - peer can now see this
session.remote_vars["prices"] = prices

# Verify it's published
session.remote_vars

## Step 7: Access Peer's Remote Variables

Now let's see how to access data shared by your peer:

In [ ]:
# View what your peer has published
session.peer_remote_vars

In [ ]:
# If peer has published 'prices', load it
# (If you're testing with yourself, you'll see your own data)
if "prices" in session.peer_remote_vars:
    peer_data = session.peer_remote_vars["prices"].load()
    print("Loaded peer's data:")
    print(peer_data)

## Step 8: Create a Privacy-Aware Function

The `@bv` decorator makes a function **Twin-aware**. When you call it with a Twin, it:
1. First runs on the **public (mock)** data for testing
2. Lets you request execution on the **private (real)** data

In [ ]:
@bv
def total_revenue(df: pd.DataFrame) -> float:
    """
    Calculate total revenue from a price DataFrame.
    Revenue = sum(units * price)
    """
    return (df["units"] * df["price"]).sum()

## Step 9: Run on Public Data

First, run the function on the public (mock) data. This is safe - no sensitive data is involved.

In [ ]:
# Call the function with the Twin
result = total_revenue(prices)

# View the result Twin
result

In [ ]:
# Access the public result (mock data)
print(f"Total revenue (mock data): ${result.public:.2f}")

## Step 10: The Private Side Requires Approval

Notice that `.private` doesn't have the answer yet - it shows a computation request instead:

In [ ]:
# Private side shows the pending computation
result.private

## Understanding Twins

A **Twin** is a dual-value container:

```
┌─────────────────────────────────────────┐
│              Twin                       │
├─────────────────┬───────────────────────┤
│  🌍 Public      │  🔒 Private           │
│  (Mock/Safe)    │  (Real/Sensitive)     │
├─────────────────┼───────────────────────┤
│  - Always       │  - Only with          │
│    accessible   │    explicit approval  │
│  - Used for     │  - Contains real      │
│    testing      │    results            │
└─────────────────┴───────────────────────┘
```

When you run a `@bv` decorated function on a Twin:
1. It runs immediately on the **public side**
2. The **private side** becomes a **request** - waiting for approval
3. Once approved, the real result populates `.private`

## Step 11: Request Private Data Execution

To get the real answer, call `.request_private()`. This sends a **computation request** to the data owner.

In [ ]:
# Request execution on private data
result.request_private()

## Step 12: Check the Session Inbox

The computation request appears in the data owner's inbox. Since we're testing with ourselves, we can see it in our own inbox:

In [ ]:
# Check inbox for pending requests
inbox = session.inbox()
inbox

In [ ]:
# Load the computation request
if len(inbox) > 0:
    request = inbox[0].load()
    request

## Step 13: Review the Code (As Data Owner)

As the data owner, you can inspect the code before running it on your private data. This is crucial for security - never run code you don't trust!

In [ ]:
# The request shows what code will be executed
# You can see the function name and parameters
if 'request' in dir() and hasattr(request, 'func'):
    print(f"Function: {request.func.__name__}")
    print(f"\nCode to review:")
    import inspect
    print(inspect.getsource(request.func))

## Step 14: Test on Mock Data First

Before running on real data, test on mock data to verify the code works correctly:

In [ ]:
# Run on mock data first (safe preview)
if 'request' in dir() and hasattr(request, 'run_mock'):
    mock_result = request.run_mock()
    print(f"Mock result: {mock_result}")

## Step 15: Run on Private Data & Approve

If you're satisfied with the code, run it on the real data and approve the result:

In [ ]:
# Run on private data
if 'request' in dir() and hasattr(request, 'run'):
    private_result = request.run()
    print(f"Private result: {private_result}")
    
    # Approve the result - this sends it back to the requester
    private_result.approve()

## Step 16: Receive the Approved Result

The approved result is now available. Load it from inbox and your original `result` variable gets updated!

In [ ]:
# Check inbox for the approved result
inbox = session.inbox()
print(f"Inbox has {len(inbox)} message(s)")

# Load all messages to update our variables
for msg in inbox:
    msg.load()

In [ ]:
# Now check the private result - it should be updated!
print(f"Total revenue (real data): ${result.private:.2f}")

## Summary

Congratulations! You've completed the Beaver Hello World tutorial. Here's what you learned:

1. **Import & Context**: `bv = beaver.ctx()` creates your gateway to Beaver

2. **Sessions**: `session = bv.active_session()` connects you to a peer

3. **Twins**: A `Twin(public=..., private=...)` keeps mock and real data together

4. **Remote Variables**: `session.remote_vars["name"] = twin` shares data (public only!)

5. **@bv Decorator**: Makes functions Twin-aware - runs on public, requests private

6. **Request Flow**:
   - Requester: `result.request_private()` → sends computation request
   - Owner: `request.run()` → `result.approve()` → sends result back
   - Requester: `inbox[...].load()` → updates original variable

### Next Steps

- Try more complex computations
- Work with real datasets (AnnData, etc.)
- Collaborate with actual peers
- Explore the BioVault app for visual dataset management

In [ ]:
# Optional: Clean up test files
# import shutil
# shutil.rmtree("./my_data", ignore_errors=True)